In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from functools import reduce
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import seaborn as sns
sns.set(style="whitegrid")

#### Read in suicide data

In [2]:
deaths = pd.read_csv('suicides_2018.txt', delimiter='\t')

#### group by county code and sum deaths that used different codes

In [3]:
county_code_total_suicides = deaths.groupby(['County Code'])['Deaths'].sum().reset_index()

In [4]:
county_code_total_suicides.head()

,County Code,Deaths
0,1003.0,33.0
1,1015.0,19.0
2,1033.0,10.0
3,1045.0,10.0
4,1049.0,10.0


#### Read in unemployment data

In [5]:
unemployment = pd.read_excel('laucnty18.xlsx', header=2)

#### Make column names mostly useful

In [6]:
unemployment.rename(columns={'Unnamed: 0':'weird_code', 'State': 'state_code', 'County': 'county_code_segment', 'Unnamed: 3': 'county_and_state', 'Unnamed: 4': 'year', 'Unnamed: 5': '?', 'Unnamed: 6': 'total_labor_force', 'Unnamed: 7': 'employed', 'Unnamed: 8': 'unemployed', 'Unemploy-': 'unemployment_rate' }, inplace=True)

In [7]:
unemployment.drop([0,1,2], inplace=True)

#### Create 'County Code' (formated to match unemployment data) from current state and county code

In [9]:
unemployment['County Code'] = unemployment['state_code'] + unemployment['county_code_segment']

In [10]:
unemployment['County Code'] = pd.to_numeric(unemployment['County Code'])

#### Isolate unemployment rate without county code

In [11]:
unemployment_rate_county_code = unemployment[['County Code', 'unemployment_rate']]